In [1]:
%matplotlib widget

import os

import numpy as np
import pandas as pd
from astropy.io import fits
from tqdm import tqdm

import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

from lvmdap._cmdline import dap
from lvmdap.analysis.plotting import plot_triang_pdfs

from pyFIT3D.common.auto_ssp_tools import load_rss

sns.set(context="talk", style="ticks", palette="colorblind", color_codes=True, font_scale=1)

import contextlib
import sys


# TAKEN FROM https://bit.ly/3fVZ36L
class DummyFile(object):
    def write(self, x): pass

@contextlib.contextmanager
def nostdout():
    save_stdout = sys.stdout
    sys.stdout = DummyFile()
    yield
    sys.stdout = save_stdout


SIMULATION_PATH = "../../_fitting-data/simulations/ssps"
OUTPUT_PATH = "../../_fitting-data/outputs"
CONFIGS_PATH = "_data/configs"

In [2]:
STELLAR_BASIS = "../../_fitting-data/_basis_mastar_v2/stellar-basis-spectra-100.fits.gz"
STELLAR_PARAMS = "../../_fitting-data/_basis_mastar_v2/stellar-basis-params-100.fits.gz"
SIGMA_INST = np.sqrt((2.62/2.355)**2 - (2.5/2.355)**2)
MASK_ELINES = "../../_fitting-data/_configs/MaNGA/mask_elines.txt"
CONFIG = "../../_fitting-data/_configs/auto_ssp_V500_several_Hb.config"
EMISSION_LINES = "../../_fitting-data/_configs/MaNGA/emission_lines_long_list.MaNGA"

# MASK_ELINES = "../../_fitting-data/_configs/mask_elines.txt"
# CONFIG = os.path.join(CONFIGS_PATH, "autodetect.auto_ssp_several.config")
# EMISSION_LINES = os.path.join(CONFIGS_PATH, "autodetect.emission_lines.txt")

In [3]:
from astropy.table import Table
import itertools as it
import scipy.optimize as so
from scipy.interpolate import CloughTocher2DInterpolator

from lvmdap.analysis.stats import weighted_pdf
from lvmdap.analysis.plotting import contours_from_pdf


# read templates
stellar_basis = fits.open(STELLAR_BASIS)
stellar_param = fits.open(STELLAR_PARAMS)

basis_params = Table(stellar_basis[1].data).to_pandas()

npars = len(stellar_param)-1
colormesh, contours, margin_pdf = [], [], []
for k in range(basis_params.index.size):
    coeffs = np.zeros(len(basis_params))
    coeffs[k] = 1.0
    margins, wPDFs, Xs, Ys = {}, {}, {}, {}
    for ihdu, (i,j) in zip(range(1,npars+1), it.combinations(range(npars),2)):
        wPDF, x_scale, y_scale = weighted_pdf(stellar_param, ihdu, coeffs=coeffs)

        if i not in margins:
            mPDF = (wPDF*stellar_param[ihdu].header["CDELT2"]).sum(axis=0)
            margins[i] = (x_scale, mPDF)
        if j not in margins:
            mPDF = (wPDF*stellar_param[ihdu].header["CDELT1"]).sum(axis=1)
            margins[j] = (y_scale, mPDF)
        
        wPDFs[ihdu] = wPDF
        Xs[ihdu], Ys[ihdu] = np.meshgrid(x_scale, y_scale)

#         wPDF_func = CloughTocher2DInterpolator(np.column_stack((X.flatten(),Y.flatten())), wPDF.flatten())
#         levels, X_, Y_, PDF_ = contours_from_pdf(
#             lambda x, y: wPDF_func(np.column_stack((x,y))),
#             range_x=x_scale[[0,-1]],
#             range_y=y_scale[[0,-1]],
#             deltas=0.05, return_grid=True
#         )
    colormesh.append((Xs, Ys, wPDFs))
#     contours.append((X_, Y_, PDF_, levels))
    margin_pdf.append(margins)

In [4]:
# read input simulated properties
input_params = pd.read_csv(os.path.join(SIMULATION_PATH, "fsps-true-params.csv"))
input_params["zlabels"] = input_params.metallicity.apply(lambda met: "{:.5f}".format(met).replace(".","p"))
input_params["tlabels"] = input_params.age.apply(lambda age: "{:.5f}gyr".format(age/1e9).replace(".","p"))
input_params["labels"] = input_params.zlabels + "_" + input_params.tlabels

# fit the simulated spectra
for name, label in tqdm(zip(input_params.name, input_params.labels), total=input_params.name.size, desc="fitting spectra", ascii=True, unit="SED"):
    target_sed = os.path.join(SIMULATION_PATH, name)
    cmdline = f"{target_sed} {STELLAR_BASIS} {SIGMA_INST} {label} {MASK_ELINES} --emission-lines-file {EMISSION_LINES} --w-range 3600 10000 --w-range-nl 3600 4700 --redshift 0 0 0 0 --sigma 0 0 0 0 --AV 0 0 0 0 -o {OUTPUT_PATH} -c".split()
    with nostdout():
        dap._main(cmdline)

fitting spectra: 100%|#################################################################################################################################| 84/84 [37:45<00:00, 26.97s/SED]


In [5]:
# read input simulated fluxes
for (tracks, library, zlabel), index in input_params.groupby(["tracks", "library", "zlabels"]).groups.items():
    el_wavelength, el_spectra, _ = load_rss(f"{SIMULATION_PATH}/fsps-ssp-{tracks}-{library}-{zlabel}-elluminosity.fits.gz")
    wavelength, stellar_spectra, _ = load_rss(f"{SIMULATION_PATH}/fsps-ssp-{tracks}-{library}-{zlabel}-stellar.fits.gz")
    wavelength, neb_emission_spectra, _ = load_rss(f"{SIMULATION_PATH}/fsps-ssp-{tracks}-{library}-{zlabel}-emission.fits.gz")
    wavelength, neb_continuum_spectra, _ = load_rss(f"{SIMULATION_PATH}/fsps-ssp-{tracks}-{library}-{zlabel}-continuum.fits.gz")
    wavelength, clean_spectra, _ = load_rss(f"{SIMULATION_PATH}/fsps-ssp-{tracks}-{library}-{zlabel}-clean.fits.gz")
    wavelength, noisy_spectra, _ = load_rss(f"{SIMULATION_PATH}/fsps-ssp-{tracks}-{library}-{zlabel}-noisy.fits.gz")
    wavelength, error_spectra, _ = load_rss(f"{SIMULATION_PATH}/fsps-ssp-{tracks}-{library}-{zlabel}-error.fits.gz")

    # read output fluxes
    wavelengths = []
    obs_joint_spectra, mod_stellar_spectra, mod_joint_spectra, obs_gas_spectra, residual_joint_spectra, obs_stellar_spectra = [], [], [], [], [], []
    fitting_colormesh, fitting_contours, fitting_margin_pdf = [], [], []
    for m in index:
        label = input_params.labels.loc[m]
        fitting = pd.read_csv(f"{OUTPUT_PATH}/coeffs_{label}", sep="\t", skiprows=1, index_col="ID", names="ID	TEFF	LOGG	META	COEFF	MinCoeff	logML	AV	NCoeff	Errs".split())
        models = fits.open(f"{OUTPUT_PATH}/output.{label}.fits.gz")

        wavelengths.append(np.array([models[0].header["CRVAL1"] + (i*models[0].header["CDELT1"]) for i in range(models[0].header["NAXIS1"])]))
        obs_joint_spectra.append(models[0].data[0])
        mod_stellar_spectra.append(models[0].data[1])
        mod_joint_spectra.append(models[0].data[2])
        obs_gas_spectra.append(models[0].data[3])
        residual_joint_spectra.append(models[0].data[4])
        obs_stellar_spectra.append(models[0].data[5])
                
        margins, wPDFs, Xs, Ys = {}, {}, {}, {}
        for ihdu, (i,j) in zip(range(1,npars+1), it.combinations(range(npars),2)):
            wPDF, x_scale, y_scale = weighted_pdf(stellar_param, ihdu, coeffs=fitting.COEFF.values)

            if i not in margins:
                mPDF = (wPDF*stellar_param[ihdu].header["CDELT2"]).sum(axis=0)
                margins[i] = (x_scale, mPDF)
            if j not in margins:
                mPDF = (wPDF*stellar_param[ihdu].header["CDELT1"]).sum(axis=1)
                margins[j] = (y_scale, mPDF)

            wPDFs[ihdu] = wPDF
            Xs[ihdu], Ys[ihdu] = np.meshgrid(x_scale, y_scale)

        fitting_colormesh.append((Xs, Ys, wPDFs))
        fitting_margin_pdf.append(margins)
    
    wavelengths = np.array(wavelengths)
    obs_joint_spectra = np.array(obs_joint_spectra)
    mod_stellar_spectra = np.array(mod_stellar_spectra)
    mod_joint_spectra = np.array(mod_joint_spectra)
    obs_gas_spectra = np.array(obs_gas_spectra)
    residual_joint_spectra = np.array(residual_joint_spectra)
    obs_stellar_spectra = np.array(obs_stellar_spectra)

In [6]:
basis_wavelength = np.array([stellar_basis[0].header["CRVAL1"] + (i*stellar_basis[0].header["CDELT1"]) for i in range(stellar_basis[0].header["NAXIS1"])])
basis_spectra = stellar_basis[0].data

labels = {"TEFF":r"$\log{T_\text{eff}}$", "LOGG":r"$\log{g}$", "MET":r"$[\text{Fe}/\text{H}]$", "ALPHAM":r"$[\alpha/\text{Fe}]$"}
npars = len(labels)
nproj = len(stellar_param) - 1
cmap = "binary_r"

# plt.close(fig="all")
fig = plt.figure(constrained_layout=True, figsize=(15,7))

n = 10
gs = GridSpec(npars, n, figure=fig)
ax0 = fig.add_subplot(gs[:, :n-npars])
ax0.set_yscale("log")
axs = []
for i in range(npars):
    axs.append([fig.add_subplot(gs[i, j+(n-npars)], sharex=(axs[i-1][j] if i!=0 else None)) for j in range(npars)])
axs = np.array(axs)
for i,j in zip(*np.triu_indices_from(axs, k=1)):
        axs[i,j].set_visible(False)

lines = []
for k in range(basis_params.index.size):
    lines += ax0.plot(basis_wavelength, basis_spectra[k], "-", color="0.8", lw=1.5)

colors_teff = sns.color_palette("rainbow_r", n_colors=basis_params.index.size)
colors_teff = np.array(colors_teff)[basis_params.sort_values("TEFF").index]

@widgets.interact(k=basis_params.index)
def update_basis(k=basis_params.index[40]):
    [ax.clear() for ax in axs.ravel()]
    plt.setp(lines, color="0.8", zorder=0)
    lines[k].set_color(colors_teff[k])
    lines[k].set_zorder(1)
    
    Xs, Ys, wPDFs = colormesh[k]
#     X_, Y_, PDF_, levels = contours[k]
    margins = margin_pdf[k]
    
    colors = []
    for ihdu, (i,j) in zip(range(1,nproj+1), it.combinations(range(npars),2)):
        X, Y, wPDF = Xs[ihdu], Ys[ihdu], wPDFs[ihdu]
        
        x_name = stellar_param[ihdu].header["CTYPE1"]
        y_name = stellar_param[ihdu].header["CTYPE2"]

        colors.append(sns.color_palette(cmap)[0])

        pcm = axs[j,i].pcolormesh(X, Y, wPDF, cmap=cmap, shading="auto")
#         axs[j,i].contour(X_, Y_, PDF_, levels=levels, colors="w", linewidths=1)

        mask_x = np.any(~np.isclose(wPDF, 0, rtol=0.05), axis=0)
        mask_y = np.any(~np.isclose(wPDF, 0, rtol=0.05), axis=1)
        axs[j,i].set_xlim(X.min(), X.max())
        axs[j,i].set_ylim(Y.min(), Y.max())
        if axs[j,i].get_subplotspec().is_last_row():
            axs[j,i].set_xlabel(labels[x_name])
        if i == 0:
            axs[j,i].set_ylabel(labels[y_name])
        else:
            axs[j,i].tick_params(labelleft=False)

    for i in range(npars):
        x, pdf = margins[i]
        axs[i,i].plot(x, pdf/pdf.max(), "-", color=colors[i])
        axs[i,i].tick_params(left=False, labelleft=False)
        sns.despine(ax=axs[i,i], left=True)

# update_basis(90)
# fig.savefig("stellar-templates.png", bbox_inches="tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='k', index=40, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1…

In [10]:
elines = {4101.0: r"$\text{H}\delta$", 4340.468: r"$\text{H}\gamma$", 4861.32: r"$\text{H}\beta$", 4958.91: r"$[\text{O}\textsc{iii}]\lambda4959$",
          5006.84: r"$[\text{O}\textsc{iii}]\lambda5007$", 6562.817: r"$\text{H}\alpha$",
          6583.6: r"$[\text{N}\textsc{ii}]\lambda6584$", 6716.47: r"$[\text{S}\textsc{ii}]\lambda6717$", 6730.85: r"$[\text{S}\textsc{ii}]\lambda6731$"}

# plt.close(fig="all")
fig = plt.figure(constrained_layout=True, figsize=(15,7))

gs = GridSpec(2, 5, figure=fig)
ax1 = fig.add_subplot(gs[0, :-1])
ax2 = fig.add_subplot(gs[1, :-1], sharex=ax1)
axins1 = fig.add_subplot(gs[0, -1:])
axins2 = fig.add_subplot(gs[1, -1:], sharex=axins1)

ax1.tick_params(labelleft=True, labelbottom=False)
axins1.tick_params(labelleft=True, labelbottom=False)
axins2.tick_params(labelleft=True, labelbottom=True)

wmin, wmax, wstep = 3600, 7500, 1.0
ax1.set_xlim(wmin,wmax)

@widgets.interact(i=range(input_params.age.drop_duplicates().size), line_wl=dict(zip(elines.values(), elines.keys())), line_width=(20, 50, wstep), kwargs=dict(continuous_update=False))
def update_spectra(i=0, line_wl=6562.817, line_width=30):
    ax1.clear()
    ax2.clear()
    axins1.clear()
    axins2.clear()

    wl = wavelengths[i]
    fl_o = obs_joint_spectra[i]
    fl_m = mod_stellar_spectra[i]
    fl_j = mod_joint_spectra[i]
    fl_g = obs_gas_spectra[i]
    fl_r = residual_joint_spectra[i]
    fl_n = obs_stellar_spectra[i]

    mask_a1 = (wmin<=wl)&(wl<=wmax)
    ax1.set_title(r"$\mathrm{Age}=%.4f\,$Gyr"%(input_params.age.drop_duplicates().values[i]/1e9))
    ax1.plot(wl[mask_a1], fl_o[mask_a1], "-", color="0.7", lw=2)
    ax1.plot(wl[mask_a1], fl_j[mask_a1], "-", color="tab:purple", lw=1.5)
    ax1.axvspan(line_wl-line_width, line_wl+line_width, fc="0.9", lw=0)
    ax1.set_ylim(0,5)
    ax1.set_ylabel(r"$F_\lambda/F_{5500}$")
    ax2.plot(wl[mask_a1], (fl_n)[mask_a1], "-", color="0.7", lw=2)
    ax2.plot(wl[mask_a1], (fl_m)[mask_a1], "-", color="tab:red", lw=1.5)
    ax2.axvspan(line_wl-line_width, line_wl+line_width, fc="0.9", lw=0)
    ax2.set_ylim(0,4)
    ax2.set_xlabel(r"$\lambda\,$[\AA]")
    ax2.set_ylabel(r"$F_\lambda/F_{5500}$")

    _, ymax = ax1.get_ylim()
    for ewl in elines:
        ax1.axvline(ewl, ls="--", lw=1, color="0.2")
        ax2.axvline(ewl, ls="--", lw=1, color="0.2")
        if ewl in [5006.84, 6583.6, 6730.85]:
            ax1.text(ewl+17, ymax-ymax*0.05, elines[ewl], ha="left", va="top", fontsize="xx-small", rotation=90, color="0.2")
        else:
            ax1.text(ewl-8, ymax-ymax*0.05, elines[ewl], ha="right", va="top", fontsize="xx-small", rotation=90, color="0.2")

    
    axins1.set_xlim(line_wl-line_width, line_wl+line_width)
    mask_ai1 = (line_wl-line_width<=wl)&(wl<=line_wl+line_width)
    axins1.plot(wl[mask_ai1], fl_o[mask_ai1], "-", color="0.7", lw=2)
    axins1.plot(wl[mask_ai1], fl_j[mask_ai1], "-", color="tab:purple", lw=1.5)
    axins2.plot(wl[mask_ai1], fl_n[mask_ai1], "-", color="0.7", lw=2)
    axins2.plot(wl[mask_ai1], fl_m[mask_ai1], "-", color="tab:red", lw=1.5)

# update_spectra()
# fig.savefig("fitting-example.png", bbox_inches="tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='i', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,…

In [11]:
basis_wavelength = np.array([stellar_basis[0].header["CRVAL1"] + (i*stellar_basis[0].header["CDELT1"]) for i in range(stellar_basis[0].header["NAXIS1"])])
basis_spectra = stellar_basis[0].data

# labels = {"TEFF":r"$\log{T_\text{eff}}$", "LOGG":r"$\log{g}$", "MET":r"$[\text{Fe}/\text{H}]$"}
# npars = len(stellar_param)-1
cmap = "binary_r"

# plt.close(fig="all")
fig = plt.figure(constrained_layout=True, figsize=(15,7))

n = 10
gs = GridSpec(npars, n, figure=fig)
ax0 = fig.add_subplot(gs[:, :n-npars])
ax0.set_yscale("log")
axs = []
for i in range(npars):
    axs.append([fig.add_subplot(gs[i, j+(n-npars)], sharex=(axs[i-1][j] if i!=0 else None)) for j in range(npars)])
axs = np.array(axs)
for i,j in zip(*np.triu_indices_from(axs, k=1)):
        axs[i,j].set_visible(False)

lines = []
for k in range(basis_params.index.size):
    lines += ax0.plot(basis_wavelength, basis_spectra[k], "-", color="0.9", lw=1.5)

colors_teff = sns.color_palette("rainbow_r", n_colors=basis_params.index.size)
colors_teff = np.array(colors_teff)[basis_params.sort_values("TEFF").index]

@widgets.interact(i=range(input_params.age.drop_duplicates().size))
def update_simulation(i=32):
    wl = wavelengths[i]
    fl_o = obs_joint_spectra[i]
    fl_m = mod_stellar_spectra[i]
    fl_j = mod_joint_spectra[i]
    fl_g = obs_gas_spectra[i]
    fl_r = residual_joint_spectra[i]
    fl_n = obs_stellar_spectra[i]
    
    [ax.clear() for ax in axs.ravel()]
    ax0.clear()
    lines = []
    for k in range(basis_params.index.size):
        lines += ax0.plot(basis_wavelength, basis_spectra[k], "-", color="0.9", lw=1.5)
    
    mask_a1 = (wmin<=wl)&(wl<=wmax)
    ax0.set_title(r"$\mathrm{Age}=%.4f\,$Gyr"%(input_params.age.drop_duplicates().values[i]/1e9))
    ax0.plot(wl[mask_a1], (fl_n)[mask_a1], "-", color="0.7", lw=2)
    ax0.plot(wl[mask_a1], (fl_m)[mask_a1], "-", color="tab:red", lw=1.5)
    ax0.set_ylim(0,6)
    ax0.set_xlabel(r"$\lambda\,$[\AA]")
    ax0.set_ylabel(r"$F_\lambda/F_{5500}$")
    
    Xs, Ys, wPDFs = fitting_colormesh[i]
    margins = fitting_margin_pdf[i]
    
    colors = []
    for ihdu, (i,j) in zip(range(1,nproj+1), it.combinations(range(npars),2)):
        X, Y, wPDF = Xs[ihdu], Ys[ihdu], wPDFs[ihdu]
        
        x_name = stellar_param[ihdu].header["CTYPE1"]
        y_name = stellar_param[ihdu].header["CTYPE2"]

        colors.append(sns.color_palette(cmap)[0])

        pcm = axs[j,i].pcolormesh(X, Y, wPDF, cmap=cmap, shading="auto")

        mask_x = np.any(~np.isclose(wPDF, 0, rtol=0.05), axis=0)
        mask_y = np.any(~np.isclose(wPDF, 0, rtol=0.05), axis=1)
        axs[j,i].set_xlim(X.min(), X.max())
        axs[j,i].set_ylim(Y.min(), Y.max())
        if axs[j,i].is_last_row():
            axs[j,i].set_xlabel(labels[x_name])
        if i == 0:
            axs[j,i].set_ylabel(labels[y_name])
        else:
            axs[j,i].tick_params(labelleft=False)

    for i in range(npars):
        x, pdf = margins[i]
        axs[i,i].plot(x, pdf/pdf.max(), "-", color=colors[i])
        axs[i,i].tick_params(left=False, labelleft=False)
        sns.despine(ax=axs[i,i], left=True)

# update_simulation(50)
# fig.savefig("fitting-example-stellar.png", bbox_inches="tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='i', index=32, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1…

In [9]:
fig, axs = plt.subplots(2, 4, figsize=(15, 7), sharex=True, sharey=True, tight_layout=True)

emin, emax = -10, +10
sns.despine(fig=fig)

@widgets.interact(i=range(input_params.age.drop_duplicates().size), line_width=(20, 50, wstep), kwargs=dict(continuous_update=False))
def update_spectra(i=12, line_width=40):
    [ax.clear() for ax in axs.ravel()]
    gas_error = (mod_joint_spectra[i] - mod_stellar_spectra[i]) - neb_emission_spectra[i]
#     gas_error = np.divide(gas_error, neb_emission_spectra[i], where=neb_emission_spectra[i]!=0, out=np.full_like(gas_error, np.nan, dtype=np.double))
#     gas_error *= 100

    for ax, line in zip(axs.ravel(), elines):
        ax.set_title(elines.get(line))
        iwl, fwl = np.argmin(np.abs(wavelength - (line-line_width))), np.argmin(np.abs(wavelength - (line+line_width)))
        ax.hist(gas_error[iwl:fwl], 20, range=(emin,emax), fc="tab:purple", lw=0, density=True, alpha=0.5)
        mean, std = np.nanmean(gas_error[iwl:fwl]), np.nanstd(gas_error[iwl:fwl])
        ax.axvspan(mean-std, mean+std, fc="0.9", lw=0, zorder=-1)
        ax.axvline(mean, ls="--", color="0.5", lw=1, zorder=-1)
        if ax.is_last_row():
            ax.set_xlabel("residual")
        ax.set_xlim(emin,emax)

# update_spectra()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='i', index=12, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1…